In [1]:
import pandas as pd
import os
import numpy as np
from ast import literal_eval

## Подготовка данных

In [2]:
root_data_dir = 'data'
shop = 'data_se'
# Тренировочные данные
data = pd.read_csv(os.path.join(root_data_dir, shop, 'orders_se_test.txt'),index_col=0) # , nrows=1000
products = pd.read_csv(os.path.join(root_data_dir, shop, 'products_se.txt'),index_col=0)
t = data['product_id']
data['name'] = t.replace(products['product_id'].values, products['name'].values)
data.to_csv('data/data_se/data_name_test.txt', index=False, columns=['order_id', 'name'])

# Тестовые данные
data = pd.read_csv(os.path.join(root_data_dir, shop, 'orders_se_train.txt'),index_col=0)
products = pd.read_csv(os.path.join(root_data_dir, shop, 'products_se.txt'),index_col=0)
t = data['product_id']
data['name'] = t.replace(products['product_id'].values, products['name'].values)
data.to_csv('data/data_se/data_name_train.txt', index=False, columns=['order_id', 'name'])

# data.head(10)

# Оставим самое необходимое

In [7]:
data_test = pd.read_csv(os.path.join(root_data_dir, shop, 'data_name_test.txt'))
data_train = pd.read_csv(os.path.join(root_data_dir, shop, 'data_name_train.txt'))
data =  pd.concat([data_test, data_train]).sort_values(by="order_id")
mas_indexet = data['name'].drop_duplicates().values
f = open('data/data_se/mas_indexet.txt', 'w', encoding='utf-8')
for i in mas_indexet:
    f.write(i + "\n")
f = open('data/data_se/data_i_test.txt', 'w')
f.write('order_id,name_id\n')
for d in data_test.values:
    # Проиндексируем продукты для простоты вычислений
    f.write(f'{d[0]},{np.where(mas_indexet == d[1])[0][0]}\n')
f.close()
f = open('data/data_se/data_i_train.txt', 'w')
f.write('order_id,name_id\n')
for d in data_train.values:
    # Проиндексируем продукты для простоты вычислений
    f.write(f'{d[0]},{np.where(mas_indexet == d[1])[0][0]}\n')
f.close()

# data.head(10)

In [8]:
data = pd.read_csv(os.path.join(root_data_dir, shop, 'data_i_test.txt'))
carts_test = data.groupby('order_id')['name_id'].apply(list).reset_index()


data = pd.read_csv(os.path.join(root_data_dir, shop, 'data_i_train.txt'))
max_i = data.name_id.max() + 1
carts_train = data.groupby('order_id')['name_id'].apply(list).reset_index()

carts = pd.concat([carts_test, carts_train]).sort_values(by=['order_id'])

# carts_test.head(10)

## Произведём подсчёт и сортировку

In [9]:
stat = np.zeros(max_i)
for a in carts['name_id'].values:
    for i in a:
        stat[i]+=1
sort_stat = stat.copy()
sort_stat.sort()

f1 = stat > 50
f2 = stat < sort_stat[-21]
f = f1*f2

## Удаляем лишнее

In [10]:
ind = []
result = open('data.txt', 'w')
result.write('order_id,name_id\n')
for i, d in enumerate(carts.values):
    t = []
    for j in d[1]:
        if f[j]:
            t.append(j)
    t = list(set(t))
    if len(t) > 1:
        for i in t:
            result.write(f'{d[0]},{i}\n')
result.close()
len_d = len(open('data.txt').readlines())
data = pd.read_csv( 'data.txt')
max_i = data.name_id.max() + 1
carts = data.groupby('order_id')['name_id'].apply(list).reset_index()

stat = np.zeros(max_i)
for a in carts['name_id'].values:
    for i in a:
        stat[i]+=1
sort_stat = stat.copy()
sort_stat.sort()

# Товары, встречающиеся меньше 50 раз
f = stat > 50
print(len_d)


290239


In [15]:
data = pd.read_csv('data.txt')
mas_indexet = data['name_id'].drop_duplicates().values
np.savetxt('data/data_se/i2.txt', mas_indexet.astype(int),  fmt = '%s')
f = open('data/data_se/data_i2.txt', 'w')
f.write('order_id,name_id\n')
for d in data.values:
    f.write(f'{d[0]},{np.where(mas_indexet == d[1])[0][0]}\n')
f.close()

# data.head(10)

,order_id,name_id
0,10,9
1,10,10
2,12,16
3,12,14
4,12,15
5,13,16
6,13,17
7,13,15
8,22,28
9,22,29


In [12]:
data = pd.read_csv(os.path.join(root_data_dir, shop, 'data_i2.txt'))
i_test = pd.read_csv(os.path.join(root_data_dir, shop, 'data_test.txt'))['order_id'].values
i_train = pd.read_csv(os.path.join(root_data_dir, shop, 'data_train.txt'))['order_id'].values
i_data = data['order_id'].values

id = np.intersect1d(i_data, i_test)
result = open('data/data_se/data_i2_test.txt', 'w', encoding='utf-8')
result.write('order_id,name_id\n')
for d in data.values:
    if d[0] in id:
        result.write(f'{d[0]},{d[1]}\n')
result.close()

id = np.intersect1d(i_data, i_train)
result = open('data/data_se/data_i2_train.txt', 'w', encoding='utf-8')
result.write('order_id,name_id\n')
for d in data.values:
    if d[0] in id:
        result.write(f'{d[0]},{d[1]}\n')
result.close()

In [ ]:
data = pd.read_csv(os.path.join(root_data_dir, shop, 'data_i2_test.txt'))
carts_test = data.groupby('order_id')['name_id'].apply(list).reset_index()
carts_test .to_csv('data/data_se/data_test.txt', index=False, columns=['order_id', 'name_id'])

data = pd.read_csv(os.path.join(root_data_dir, shop, 'data_i2_train.txt'))
carts_train = data.groupby('order_id')['name_id'].apply(list).reset_index()
carts_train.to_csv('data/data_se/data_train.txt', index=False, columns=['order_id', 'name_id'])

data = pd.read_csv(os.path.join(root_data_dir, shop, 'data_i2.txt'))
carts = data.groupby('order_id')['name_id'].apply(list).reset_index()
carts.to_csv('data/data_se/data.txt', index=False, columns=['order_id', 'name_id'])
